In [2]:
import os
import json
import uuid
import requests
from typing import Dict, Any, TypedDict, Optional

ACCESS_TOKEN = "IGAAx77ZB5EqXBBZAFA4eGFtRUUxREFIa3B6T184aFk2RTFfcnYzQmkxUzJTUW5nV3dieTk0V251Yld5ZAnNPbHhDcDJrTGcybno5dEdYSDd5NHdXUWdvaUhfdkhrdVJFWDRLWGN2bWg0RlAwQjJMcm11RkxlXzBJTldmR2R1RFZA2UQZDZD"

In [ ]:
class ErrorResponse(TypedDict):
    error: str
    status_code: int
    details: Optional[Dict[str, Any]]

class InstagramHelper:
    
    def __init__(self, access_token: str | None = None) -> None:
        
        if not access_token:
            raise ValueError("Tienes que ingresar el access_token.")
        
        self.access_token: str = access_token
        self.URL_BASE = "https://graph.instagram.com/"

    def get(self, url: str) -> Dict[str, Any] | ErrorResponse:
        """
        Realiza una petición GET y retorna los datos o un error descriptivo.
        
        Args:
            url (str): URL a consultar.
        
        Returns:
            Dict[str, Any]: Datos JSON si la respuesta es exitosa (200).
            ErrorResponse: Diccionario con detalles del error si falla.
                Ejemplo: {"error": "HTTP Error", "status_code": 404, "details": None}
        """
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Lanza HTTPError para códigos 4XX/5XX
            return response.json()
        
        except requests.exceptions.RequestException as e:
            error_data: ErrorResponse = {
                "error": str(e),
                "status_code": getattr(e.response, "status_code", 500),
                "details": None,
            }
            if isinstance(e, requests.exceptions.HTTPError):
                try:
                    error_data["details"] = e.response.json()  # Si la API devuelve JSON en el error
                except ValueError:
                    error_data["details"] = {"message": e.response.text}
            return error_data
        
    def dowload_file(self, url: str, path: str) -> str | None:
        
        
        new_url, _ = url.split("?")
        file_name = new_url.split("/")[-1]
        new_path = f"{path}/{file_name}"
        
        if os.path.exists(new_path):
            print("el archivo ya existe")
            return new_path
            
        try:
            # Realizar la petición GET
            response = requests.get(url, stream=True, timeout=10)
            response.raise_for_status()  # Verificar si hubo errores HTTP

            # Guardar la imagen en modo binario
            with open(new_path, "wb") as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
                    
            return new_path

        except Exception as e:
            print(f"Error al descargar la imagen: {e}")
            return None
    
    def make_json(self, data: Dict[str, Any], path: str) -> None:
                
        with open(path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        
        return
    
    def get_user_simple_deatil(self) -> Dict[str, Any] | ErrorResponse:
        
        url_base = f"{self.URL_BASE}me?fields=id,username,profile_picture_url,account_type,followers_count,media_count&access_token={self.access_token}"
        
        data = self.get(url_base)
        
        return data

    #3
    def get_user_followers(self) -> list[dict[str, Any]] | ErrorResponse:
        
        url_base = f"{self.URL_BASE}me?fields=fields=followers_count,follows_count&access_token={self.access_token}"
        
        data = self.get(url_base)
        
        return data
        

    def get_user_post(self, dowload_media_file: bool = False, make_json: bool = False) -> Dict[str, Any] | ErrorResponse:
        
        url_base = f"{self.URL_BASE}me/media?fields=id,caption,media_type,media_url,permalink,timestamp&access_token={self.access_token}"
        
        data = self.get(url_base)
        
        if "error" in data:
            return data
        else:
            data = data["data"]
        
        if dowload_media_file:
            
            user_detail = self.get_user_simple_deatil()
            
            if "error" in user_detail:
                dir_name = str(uuid.uuid4())
            else:
                dir_name = user_detail['id']
            
            new_path = f"images/{dir_name}"
            os.makedirs(new_path, exist_ok=True)
            
            new_data: list[dict[str, Any]] = []
            
            for dt in data:
                
                file_name = self.dowload_file(dt["media_url"], new_path)
                dt["file_name"] = file_name
                new_data.append(dt)
                
            else:
                data = new_data
        else:
            new_data: list[dict[str, Any]] = data
        
        
        if make_json:
            
            file_name = str(uuid.uuid4()) + ".json"
            path = f"json/{file_name}"
            
            print(path)
            self.make_json(data, path)
        
        return data

In [22]:
instagram = InstagramHelper(ACCESS_TOKEN)

In [24]:
instagram.get_user_post(make_json=True, dowload_media_file=True)


el archivo ya existe
el archivo ya existe
el archivo ya existe
el archivo ya existe
el archivo ya existe
el archivo ya existe
json/9560df50-7637-4577-a59f-7d4ea933513e.json


[{'id': '18029394026450741',
  'caption': "weyu;dswuer5p[y'lbvdrerotpy\n\nj\n7848",
  'media_type': 'IMAGE',
  'media_url': 'https://scontent.cdninstagram.com/v/t51.75761-15/500034735_17842504635498107_7676378034073085496_n.jpg?stp=dst-jpg_e35_tt6&_nc_cat=107&ccb=7-5&_nc_sid=18de74&_nc_ohc=vCoyXcrHInIQ7kNvwEEamgG&_nc_oc=Adk3xgMq9qqIA3LE_MzxhyZgYq3iUB5kCuzUgxQYM9MMq25cmCjF95FVVwEkn_bGeMI&_nc_zt=23&_nc_ht=scontent.cdninstagram.com&edm=ANo9K5cEAAAA&_nc_gid=FNBAVDb5mMsOqfVN_agvhA&oh=00_AfKOSnR1oFfKxydqnXFNHNO3O_xY2waKfRluIT80-h8a_w&oe=68340B25',
  'permalink': 'https://www.instagram.com/p/DJ68c_DOVk1/',
  'timestamp': '2025-05-21T15:41:26+0000',
  'file_name': 'images/9917490378343382/500034735_17842504635498107_7676378034073085496_n.jpg'},
 {'id': '17932013745028773',
  'caption': 'un raton todo gordo y pendejo',
  'media_type': 'IMAGE',
  'media_url': 'https://scontent.cdninstagram.com/v/t51.75761-15/500088863_17842504593498107_27745750770968668_n.jpg?stp=dst-jpg_e35_tt6&_nc_cat=107&ccb=